In [2]:
import numpy as np
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [1]:
#@title Change directory for jupyter notebook
import os
print(os.getcwd())
print(os.chdir('/content/drive/MyDrive/5sem/2nd_GR/Knapsack-GNN/Knapsack_jupyter_notebook'))

/content
None


# Dataset preparation (incl. normalization)
Using pytorch dataloader framework to standard deep learning dataset e.g. minibatch

As normalization is essential part of machine learning algorithm to make model learn generalizable.

Normalization is done by:\
⋅ Divide the prices by the maximum price of the problem.\
⋅ Divide the weights by the capacity.\
⋅ Remove the capacity from the inputs as it is embedded in the weights now.

In [4]:
class Dataset_create_knapsack(Dataset):
  def __init__(self, count, item_count=5):
    self.x = [[], [], []]    # list elements: x_weights, x_prices, x_capacity
    self.y = []              # y_best_picks
    for _ in range(count):
        p = self.create_knapsack(item_count)
        self.x[0].append(p[0])  # x_weights
        self.x[1].append(p[1])  # x_prices
        self.x[2].append(p[2])  # x_capacity
        self.y.append(p[3])
    self.x = [torch.Tensor(x) for x in self.x]          # make list to torch type
    self.x_cap = self.x[2].unsqueeze(1)
    print(self.x_cap.shape)
    self.x = torch.stack((self.x[0],self.x[1]), dim=1)  # torch.Size([10000, 2, 5]) # stacking s.t. batch_size, (weights + prices), len(item) + len(capacity)
    # self.x_ic = torch.stack((self.x, self.x_cap), dim=2)
    self.y = torch.Tensor(self.y)                       # torch.Size([10000, 5])

    # Data preprocessing: Normalizing the input
    self.x[:,0,:] /= self.x_cap          # x_weights_norm = x_weights / x_capacity
    # print("x is", self.x[:,1,:].shape)
    # denom = torch.max(self.x[:,1,:], dim=1)
    denom = self.x[:,1,:].max(dim=1)
    self.x[:,1,:] /= denom[0][:,None]         # x_prices_norm = x_prices / np.max(x_prices)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

  def __len__(self):
    return len(self.y)

  def create_knapsack(self, item_count=5): 
    x_weights = np.random.randint(1, 15, item_count)
    x_prices = np.random.randint(1, 10, item_count)
    x_capacity = np.random.randint(15, 50)
    _, y_best_picks = brute_force_knapsack(x_weights, x_prices, x_capacity)
    return x_weights, x_prices, x_capacity, y_best_picks


Also the algorithm basis is the brute-force knapsack. i.e. trying all possible combinations of items to find the best solution outputs, picks.

where binary selection meaning 0 for not selecting 1 for selecting the item:

In [5]:
def brute_force_knapsack(x_weights, x_prices, x_capacity):
    picks_space = 2 ** x_weights.shape[0] ## to pick it or not to pick it (binary)
    best_price = 0
    best_picks = None
    for p in range(picks_space):
        picks = np.zeros((x_weights.shape[0]))
        for i, c in enumerate("{0:b}".format(p)[::-1]):
          # This loop generates all possible combinations of items to pick or not
          #  to pick by converting the binary representation of p to a list of
          #  0s and 1s, and assigning the result to picks.
            picks[i] = c
        price, violation = test_knapsack(x_weights, x_prices, x_capacity, picks)
        if violation == 0:
            if price > best_price:
                best_price = price
                best_picks = picks
    return best_price, best_picks
    
def test_knapsack(x_weights, x_prices, x_capacity, picks):
    total_price = np.dot(x_prices, picks)
    total_weight = np.dot(x_weights, picks)
    return total_price, max(total_weight - x_capacity, 0)

In [6]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

total_samples = 10000
# Parameters
params = {'batch_size': 64,
          'max_epochs': 100,
           'metric': "sv"}
          #'n_iter': 100}        # iteration is already determined by batch_size s.t. total_samples / params['batch_size'] => 151 batches in total
knapsack_dataset = Dataset_create_knapsack(total_samples)   # create 10000 samples
knapsack_dataloader = DataLoader(dataset=knapsack_dataset, batch_size=params['batch_size'], shuffle=True, num_workers=2)

data_iter = iter(knapsack_dataloader)
data = next(data_iter)    # next iteration (next batch)
features, labels = data
n_iter = round(total_samples / params['batch_size'])

torch.Size([10000, 1])


<ipython-input-4-6dfd6b76df20>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  self.x = [torch.Tensor(x) for x in self.x]          # make list to torch type


# Motivation
Since the knapsack problem cannot be solved in polynomial time, it's NP-hard. i.e. computationally intractable and cannot be solved in polynomial time for all cases, and there is no known effcient algorithm for solving in all cases.

Hence, an approach with neural network to learn the brute-force with much shorter inference time could lead to optimal solution without having to seek all combinations of solution. e.g. number of items is infinitely large (1000 items to run brute-force algorithm)

# Supervised learning approach

Working principle for the model is simple as single-layer neural network:\
Firstly, concatenate two inputs (weights, prices) and put them into a single hidden layer. Then it predicts the optimal combination prediction from the given inputs.\
Model architecture is following

<img src="img/model_architecture" alt="img/model_architecture" />

# Metrics

⋅ **overpricing**: evaluating the average difference between the prices of the chosen items and the optimum solution (from neural network prediction)\
⋅ **space violation**: positive difference between the sum of the chosen items' weights and the knapsack's capacity

In [7]:
class SupervisedContinuesKnapsack(nn.Module):
    def __init__(self, item_count=5):
        super(SupervisedContinuesKnapsack, self).__init__()
        self.item_count = item_count
        self.fc = nn.Linear(item_count*2, item_count, bias=False)

    def forward(self, x_weights, x_prices):
        inputs_concat = torch.cat([x_weights, x_prices], dim=1) # torch.Size(["batch_size", 2 * 5])
        picks = torch.sigmoid(self.fc(inputs_concat))
        # print("picks:", picks)
        return picks

    # Evaluation metrices
    def metric_overpricing(self, y_true, y_pred):
        y_pred = torch.round(y_pred)
        return torch.mean(torch.sum(y_pred * y_true[:, 1].reshape(-1, 1), -1) - torch.sum(y_true[:, 1] * y_true[:, 0], -1))


    # def metric_space_violation(input_weights, input_capacity):
    #     def space_violation(y_true, y_pred):
    #         y_pred = K.round(y_pred)
    #         return K.mean(K.maximum(K.batch_dot(y_pred, input_weights, 1) - input_capacity, 0))

    #     return space_violation


    def metric_space_violation(self, x_weights, y_true, y_pred, input_capacity):
        y_pred = torch.round(y_pred)
        y_pred = y_pred.unsqueeze(dim=-1) # add an extra dimension to y_pred
        violation = torch.matmul(y_pred, x_weights.unsqueeze(dim=1)) - input_capacity
        violation = torch.clamp(violation, 0, None) # clamp all negative values to 0
        return torch.mean(violation)

    def metric_pick_count(self, y_true, y_pred):
        y_pred = torch.round(y_pred)
        return torch.mean(torch.sum(y_pred, dim=1) - torch.sum(y_true[:,0], dim=1))

# Train the model
def train_knapsack(model, dataloader, metric, n_epochs):
    train_results = []
    if metric == "BCE":
        criterion = nn.BCELoss()
    elif metric == "op":
        criterion = model.metric_overpricing
    elif metric == "sv":
        criterion = model.metric_space_violation
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_loss = float("inf")

    if os.path.exists("best_model_saved/best_model_sck.pth"): os.remove("best_model_saved/best_model_sck_" + str(metric) + ".pth")
    for epoch in range(n_epochs):
        for i, data in enumerate(dataloader):   # Iterating through the minibatches of the data
            # data is a tuple of (input_features, labels)
            input_features, labels = data
            x_weights = input_features[:,0,:]
            x_prices  = input_features[:,1,:]
            # print("x_weights",x_weights.shape)
            # print("x_prices",x_prices.shape)
            ## Forward pass
            if (i+1) % n_iter == 0:
                print(f"epoch {epoch+1}/{n_epochs}, step {i+1}/{n_iter}, best_loss {best_loss:.2f}")
            train_picks = model(x_weights, x_prices)
            if metric == "sv":  
                train_loss = criterion(x_weights, train_picks, labels, )
                train_loss = torch.autograd.Variable(train_loss, requires_grad=True)
            else:
                train_loss = criterion(train_picks, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            ## Save the best model
            if train_loss < best_loss:
                torch.save(model.state_dict(), "best_model_saved/best_model_sck_" + str(metric) + ".pth")
                best_loss = train_loss
        # print(f"Best_loss: {best_loss:.2f}")
        train_results.append(best_loss)


Now we train the model with given parameter values. In this case, we test it with overpricing as metric.

In [ ]:
# Load the best model and evaluate
# model.load_state_dict(torch.load("best_model.pth"))
model_knapsack = SupervisedContinuesKnapsack()
train_knapsack(model_knapsack, knapsack_dataloader, metric = params['metric'], n_epochs=params['max_epochs'])

epoch 1/100, step 156/156, best_loss 0.00
epoch 2/100, step 156/156, best_loss 0.00
epoch 3/100, step 156/156, best_loss 0.00
epoch 4/100, step 156/156, best_loss 0.00
epoch 5/100, step 156/156, best_loss 0.00
epoch 6/100, step 156/156, best_loss 0.00
epoch 7/100, step 156/156, best_loss 0.00
epoch 8/100, step 156/156, best_loss 0.00
epoch 9/100, step 156/156, best_loss 0.00
epoch 10/100, step 156/156, best_loss 0.00
epoch 11/100, step 156/156, best_loss 0.00
epoch 12/100, step 156/156, best_loss 0.00
epoch 13/100, step 156/156, best_loss 0.00
epoch 14/100, step 156/156, best_loss 0.00
epoch 15/100, step 156/156, best_loss 0.00
epoch 16/100, step 156/156, best_loss 0.00
epoch 17/100, step 156/156, best_loss 0.00
epoch 18/100, step 156/156, best_loss 0.00
epoch 19/100, step 156/156, best_loss 0.00
epoch 20/100, step 156/156, best_loss 0.00
epoch 21/100, step 156/156, best_loss 0.00
epoch 22/100, step 156/156, best_loss 0.00
epoch 23/100, step 156/156, best_loss 0.00
epoch 24/100, step 1

#Supervised discrete solution with hidden layer.

Problem with the above approach is the round function cannot be trainable for the model. i.e. when it backpropagates, the gradient cannot pass through the round function. Thus, the model won't be trainable.

To deal with that, we need continuous function that is trainable(differentiable in numerical sense) and has outputs only ones and zeros.

In this sense, the proper activation function would be chosen are $tanh$ and σ activations.

This can be formulated by following:
$f(x) = (tanh(W_{1}⋅x) + σ(W_{2}⋅x))^2$
where $W_{i}$ are weight matrices.

In [ ]:
class SupervisedDiscreteKnapsack(nn.Module):
    def __init__(self, item_count=5):
        super(SupervisedDiscreteKnapsack, self).__init__()
        self.item_count = item_count
        self.linear1 = nn.Linear(item_count*2, item_count)
        self.linear2 = nn.Linear(item_count, item_count)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x_weights, x_prices):
        inputs_concat = torch.cat([x_weights, x_prices], dim=1) # torch.Size(["batch_size", 2 * 5])
        x = self.linear1(inputs_concat)
        x = self.tanh(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        x = x * x
        return x
        
    # Evaluation metrices
    def metric_overpricing(self, y_true, y_pred):
        y_pred = torch.round(y_pred)
        return torch.mean(torch.sum(y_pred * y_true[:, 1].reshape(-1, 1), -1) - torch.sum(y_true[:, 1] * y_true[:, 0], -1))

    def metric_space_violation(self, x_weights, y_true, y_pred):
        y_pred = torch.round(y_pred)
        y_pred = y_pred.unsqueeze(dim=-1) # add an extra dimension to y_pred
        violation = torch.matmul(y_pred, x_weights.unsqueeze(dim=1)) - 1
        violation = torch.clamp(violation, 0, None) # clamp all negative values to 0
        return torch.mean(violation)

    def metric_pick_count(self, y_true, y_pred):
        y_pred = torch.round(y_pred)
        return torch.mean(torch.sum(y_pred, dim=1) - torch.sum(y_true[:,0], dim=1))

# Train the model
def train_knapsack(model, dataloader, metric, n_epochs):
    train_results = []
    if metric == "BCE":
        criterion = nn.BCELoss()
    elif metric == "op":
        criterion = model.metric_overpricing
    elif metric == "sv":
        criterion = model.metric_space_violation
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_loss = float("inf")

    if os.path.exists("best_model_saved/best_model_sck.pth"): os.remove("best_model_saved/best_model_sdk_" + str(metric) + ".pth")
    for epoch in range(n_epochs):
        for i, data in enumerate(dataloader):   # Iterating through the minibatches of the data
            # data is a tuple of (input_features, labels)
            input_features, labels = data
            x_weights = input_features[:,0,:]
            x_prices  = input_features[:,1,:]
            ## Forward pass
            if (i+1) % n_iter == 0:
                print(f"epoch {epoch+1}/{n_epochs}, step {i+1}/{n_iter}, best_loss {best_loss:.2f}")
            train_picks = model(x_weights, x_prices)
            if metric == "sv":  
                train_loss = criterion(x_weights, train_picks, labels)
                train_loss = torch.autograd.Variable(train_loss, requires_grad=True)
            else:
                train_loss = criterion(train_picks, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            ## Save the best model
            if train_loss < best_loss:
                torch.save(model.state_dict(), "best_model_saved/best_model_sdk_" + str(metric) + ".pth")
                best_loss = train_loss
        # print(f"Best_loss: {best_loss:.2f}")
        train_results.append(best_loss)

# Load the best model and evaluate
# model.load_state_dict(torch.load("best_model.pth"))
model_knapsack = SupervisedDiscreteKnapsack()
train_knapsack(model_knapsack, knapsack_dataloader, "BCE", params['max_epochs'])

epoch 1/100, step 156/156, best_loss 0.45
epoch 2/100, step 156/156, best_loss 0.38
epoch 3/100, step 156/156, best_loss 0.35
epoch 4/100, step 156/156, best_loss 0.32
epoch 5/100, step 156/156, best_loss 0.28
epoch 6/100, step 156/156, best_loss 0.28
epoch 7/100, step 156/156, best_loss 0.28
epoch 8/100, step 156/156, best_loss 0.28
epoch 9/100, step 156/156, best_loss 0.28
epoch 10/100, step 156/156, best_loss 0.28
epoch 11/100, step 156/156, best_loss 0.28
epoch 12/100, step 156/156, best_loss 0.28
epoch 13/100, step 156/156, best_loss 0.28
epoch 14/100, step 156/156, best_loss 0.28
epoch 15/100, step 156/156, best_loss 0.28
epoch 16/100, step 156/156, best_loss 0.28
epoch 17/100, step 156/156, best_loss 0.27
epoch 18/100, step 156/156, best_loss 0.24
epoch 19/100, step 156/156, best_loss 0.24
epoch 20/100, step 156/156, best_loss 0.24
epoch 21/100, step 156/156, best_loss 0.24
epoch 22/100, step 156/156, best_loss 0.24
epoch 23/100, step 156/156, best_loss 0.24
epoch 24/100, step 1

# Result (training loss of both models)
Result will roughly be:
⋅ one hidden layer (BCELoss): 

epoch 100/100, step 100/100, best_loss 0.18\
⋅ overpricing loss\
epoch 100/100, step 156/156, best_loss -60.61

Supervised with hidden layer with continuous activation function approach: 

⋅ MLP (BCELoss)
epoch 100/100, step 100/100, best_loss 0.13

# Two fundamental problems with the supervised approach: 
1. optimum solution (so we need to find hyper-parameter tuning via grid or random search to find optimal solution first) is mendatory to start training
2. No control over space violation and overpricing.

Those two above can be dealt in unsupervised approach.

# Training NN in unsupervised fashion

We can train the model without computing optimal solution by unsupervised approach:

Cost function composes of two:
1. $Totalprice = p · o$
2. $SpaceViolation = max(w · o - c, 0)$
We firstly maximize the sum of chosen item's prices, which is Totalprice. Then the picks should not surpass the knapsack's capacity. As we mentioned the problem with the supervised approach, we want to control the cost function with SpaceViolation. To do this, λ coefficent is multiplied to it:
That is, \\
$J = -TotalPrice + λSpaceViolation$

where o:= output, w := input weights, c := input capacity, p:= input price

In [ ]:
class unsupervised_discrete_knapsack(nn.Module):
    def __init__(self, item_count=5):
        super(unsupervised_discrete_knapsack, self).__init__()
        self.item_count = item_count
        self.linear1 = nn.Linear(item_count*2, item_count)
        self.linear2 = nn.Linear(item_count, item_count)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x_weights, x_prices):
        inputs_concat = torch.cat([x_weights, x_prices], dim=1) # torch.Size(["batch_size", 2 * 5])
        # print(inputs_concat.shape)
        x = self.linear1(inputs_concat)
        x = self.tanh(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        x = x * x
        return x
        
    # Evaluation metrices
    def metric_unsupervised(self, input_weights, input_prices, y_pred, input_capacity=5, lmda=0.001):  # y_true == input_weights, input_prices, input_capacity
        picks = y_pred
        # input_weights, input_prices = y_true
        TP = torch.sum(picks * input_prices, 1) # total price   ## element-wise product
        SV = torch.sum(picks * input_weights, 1) - input_capacity # space violation
        return (-1 * TP) + lmda * torch.maximum(SV, 0)  ## torch.zeros_like(SV)

# Train the model
def train_knapsack(model, dataloader, n_epochs, metric="usp"):
    train_results = []
    criterion = model.metric_unsupervised
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_loss = float("inf")

    if os.path.exists("best_model_saved/best_model_usp.pth"): os.remove("best_model_saved/best_model_" + str(metric) + ".pth")
    for epoch in range(n_epochs):
        for i, data in enumerate(dataloader):   # Iterating through the minibatches of the data
            input_features, labels = data       # data is a tuple of (input_features, labels)
            x_weights = input_features[:,0,:]
            x_prices  = input_features[:,1,:]
            ## Forward pass
            if (i+1) % 10 == 0:
                print(f"epoch {epoch+1}/{n_epochs}, step {i+1}/{n_iter_usp}, best_loss {best_loss:.2f}")
            # train_picks = model(x_weights, x_prices)
            train_loss = model(x_weights, x_prices)
            mean_loss = train_loss.mean()
            optimizer.zero_grad()
            mean_loss.backward()
            optimizer.step()
            ## Save the best model
            if mean_loss < best_loss:
                torch.save(model.state_dict(), "best_model_saved/best_model_" + str(metric) + ".pth")
                best_loss = mean_loss
        print(f"Best_loss: {best_loss:.2f}")
        train_results.append(best_loss)

# Parameters
params_usp = {'batch_size': 64,
              'max_epochs': 100}
n_iter_usp = round(total_samples / params_usp['batch_size'])

# Load the best model and evaluate
# model.load_state_dict(torch.load("best_model.pth"))
knapsack_dataloader = DataLoader(dataset=knapsack_dataset, batch_size=params_usp['batch_size'], shuffle=True, num_workers=2)    ## update dataloader as batch_size is changed
model_knapsack = unsupervised_discrete_knapsack()
train_knapsack(model_knapsack, knapsack_dataloader, params_usp['max_epochs'])

epoch 1/100, step 10/156, best_loss 0.15
epoch 1/100, step 20/156, best_loss 0.06
epoch 1/100, step 30/156, best_loss 0.02
epoch 1/100, step 40/156, best_loss 0.01
epoch 1/100, step 50/156, best_loss 0.01
epoch 1/100, step 60/156, best_loss 0.00
epoch 1/100, step 70/156, best_loss 0.00
epoch 1/100, step 80/156, best_loss 0.00
epoch 1/100, step 90/156, best_loss 0.00
epoch 1/100, step 100/156, best_loss 0.00
epoch 1/100, step 110/156, best_loss 0.00
epoch 1/100, step 120/156, best_loss 0.00
epoch 1/100, step 130/156, best_loss 0.00
epoch 1/100, step 140/156, best_loss 0.00
epoch 1/100, step 150/156, best_loss 0.00
Best_loss: 0.00
epoch 2/100, step 10/156, best_loss 0.00
epoch 2/100, step 20/156, best_loss 0.00
epoch 2/100, step 30/156, best_loss 0.00
epoch 2/100, step 40/156, best_loss 0.00
epoch 2/100, step 50/156, best_loss 0.00
epoch 2/100, step 60/156, best_loss 0.00
epoch 2/100, step 70/156, best_loss 0.00
epoch 2/100, step 80/156, best_loss 0.00
epoch 2/100, step 90/156, best_loss

In [ ]:
# Load the best model and evaluate
model_udk = unsupervised_discrete_knapsack()
model_udk.load_state_dict(torch.load("best_model_saved/best_model_usp.pth"))
model_sck = SupervisedContinuesKnapsack()
model_sck.load_state_dict(torch.load("best_model_saved/best_model_sck.pth"))
model_sdk = SupervisedDiscreteKnapsack()
model_sdk.load_state_dict(torch.load("best_model_saved/best_model_sdk.pth"))

# Example input data
# print(x_weights.shape)
# print(x_prices.shape)
# x_weights = torch.Tensor([2, 3, 4, 5, 6])
# x_prices = torch.Tensor([3, 4, 5, 6, 2])
x_weights = np.random.randint(9, size=(5))
x_prices = np.random.randint(9, size=(5))
x_capacity = np.random.randint(9)

print("x_weights: ", x_weights)
print("x_prices: ", x_prices)
print("x_capacity: ", x_capacity)

best_price, best_picks = brute_force_knapsack(x_weights, x_prices, x_capacity)
print("__Brute_force:")
print("best_price: ", best_price)
print("best_picks: ", best_picks)

x_weights = torch.from_numpy(x_weights).unsqueeze(0).float()    # weight tensors are float dtype
x_prices  = torch.from_numpy(x_prices).unsqueeze(0).float()
# print(x_prices.shape)
# Use the model to generate predictions
with torch.no_grad():
    picks_udk = model_udk(x_weights, x_prices)
    picks_sck = model_sck(x_weights, x_prices)
    picks_sdk = model_sdk(x_weights, x_prices)

# Convert predictions to binary (1 if item is picked, 0 if not)
picks_binary_udk = torch.round(picks_udk).squeeze().numpy()
picks_binary_sck = torch.round(picks_sck).squeeze().numpy()
picks_binary_sdk = torch.round(picks_sdk).squeeze().numpy()

print("Prediction_udk (binary):", picks_binary_udk)
print("Prediction_sck (binary):", picks_binary_sck)
print("Prediction_sdk (binary):", picks_binary_sdk)


x_weights:  [3 3 1 2 2]
x_prices:  [1 5 5 5 1]
x_capacity:  3
__Brute_force:
best_price:  10.0
best_picks:  [0. 0. 1. 1. 0.]
Prediction_udk (binary): [0. 0. 0. 0. 0.]
Prediction_sck (binary): [0. 1. 1. 1. 0.]
Prediction_sdk (binary): [0. 0. 0. 1. 0.]


In [ ]:
TestFCL = nn.Linear(10,5)
inputs_concat = torch.cat([x_weights, x_prices], dim=1).float()
input = torch.randint(1,9,(128, 10))
print(inputs_concat.shape)
print(inputs_concat)
print(input.shape)
output = TestFCL(inputs_concat)
print(output)

torch.Size([1, 10])
tensor([[0., 1., 4., 8., 1., 0., 3., 4., 7., 4.]])
torch.Size([128, 10])
tensor([[-1.9526, -4.1712,  1.0911,  3.3820, -1.3542]],
       grad_fn=<AddmmBackward0>)


In [ ]:
#@title Now let's compare



In [ ]:
# #@title Example
# import torch

# # input data
# input_weights = torch.Tensor([[5, 4, 6, 2, 1], [3, 5, 7, 2, 1]])
# input_prices = torch.Tensor([[10, 20, 30, 40, 50], [5, 10, 15, 20, 25]])
# input_capacity = torch.Tensor([10, 15])
# cvc = 1
# # create target
# target = torch.Tensor([[1, 1, 1, 0, 0], [0, 1, 1, 1, 0]])

# # create picks (prediction)
# picks = torch.Tensor([[1, 1, 0, 0, 0], [0, 0, 1, 1, 0]])

# # calculate loss
# loss_fn = knapsack_loss(input_weights, input_prices, input_capacity, cvc)
# loss = loss_fn(picks, target)
# print(loss)

In [ ]:
# A = (-1 * torch.sum(picks * input_prices, 1))
# B = torch.sum(picks * input_weights, 1) - input_capacity
# C = torch.maximum(B, torch.zeros_like(B))
# print(B)
# print(C)
# B = A>0
# # B = torch.max(A, 0)[0]
# print(B)
# B[0]

In [ ]:
######################
# def unsupervised_discrete_knapsack(item_count=5):
#     input_weights = Input((item_count,))
#     input_prices = Input((item_count,))
#     input_capacity = Input((1,))
#     inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
#     concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
#     concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
#     concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
#     picks = Multiply()([concat_multiply, concat_multiply])
#     model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
#     model.compile("sgd",
#                   knapsack_loss(input_weights, input_prices, input_capacity, 1),
#                   metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
#                            metric_overprice(input_prices), metric_pick_count()])
#     return model
# model = unsupervised_discrete_knapsack()
# train_knapsack(model)

In [ ]:
# ######################

# class SupervisedContinuesKnapsackOneHidden(nn.Module):
#     def __init__(self, item_count=5):
#         super(SupervisedContinuesKnapsackOneHidden, self).__init__()
#         self.item_count = item_count
#         # self.fc1 = nn.Linear(3*item_count, item_count*10)
#         # self.fc2 = nn.Linear(item_count*10, item_count)

#         self.input_weights = nn.Linear(item_count, item_count*10)
#         self.input_prices = nn.Linear(item_count, item_count*10)
#         self.input_capacity = nn.Linear(1, item_count*10)
#         self.picks = nn.Linear(item_count*10, item_count)
        
#     def forward(self, input_weights, input_prices, input_capacity):
#         inputs_concat = torch.cat((input_weights, input_prices, input_capacity), dim=-1)
#         picks = self.input_weights(inputs_concat)
#         picks = self.input_prices(picks)
#         picks = self.input_capacity(picks)
#         picks = self.picks(picks)
#         return picks
    
# model = SupervisedContinuesKnapsackOneHidden()


# # Train the model
# def train_knapsack_DL(model, train_x, train_y, test_x, test_y):
#   # Define loss function and optimizer
#   criterion = nn.BCELoss()
#   optimizer = optim.SGD(model.parameters(), lr=0.01)
#   best_loss = float("inf")

#   # Prepare data for training
#   train_dataset = TensorDataset(train_x, train_y)
#   train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

#   for epoch in range(96):
#       for input_weights, input_prices, input_capacity, label in train_loader:
#           # Forward pass
#           output = model(input_weights, input_prices, input_capacity)
#           # Compute the loss
#           loss = criterion(output, label)
#           # Zero the gradients
#           optimizer.zero_grad()
#           # Backward pass and optimize
#           loss.backward()
#           optimizer.step()
#           # Save the best model
#           if loss < best_loss:
#               torch.save(model.state_dict(), "best_model.pth")
#               best_loss = loss
#               # save the best model
#               torch.save(model.state_dict(), "best_model.pth")
#       model.load_state_dict(torch.load("best_model.pth"))
#       model.eval()

#       train_loss = criterion(model(train_x), train_y)
#       test_loss = criterion(model(test_x), test_y)

#       print("Model results(Train/Test):")
#       print(f"Loss:               {train_loss:.2f} / {test_loss:.2f}")
#       train_picks =   model(train_x)
#       test_picks =    model(test_x)
#       train_space_violation = torch.sum(torch.mul(train_x[0], train_picks))
#       test_space_violation = torch.sum(torch.mul(test_x[0], test_picks))

#       train_overprice = torch.sum(torch.mul(torch.mul(train_x[1], train_picks), train_picks))
#       test_overprice = torch.sum(torch.mul(torch.mul(test_x[1], test_picks), test_picks))

#       train_pick_count = torch.sum(train_picks)
#       test_pick_count = torch.sum(test_picks)

#       train_results = evaluate_model(model, train_x, train_y, 64, 0)
#       test_results = evaluate_model(model, test_x, test_y, 64, 0)


In [ ]:
# # Test
# # Data preprocessing: Normalizing the input
# x_prices_norm = x_prices / np.max(x_prices)
# x_weights_norm = x_weights / x_capacity

# print("Weights:", x_weights)
# print("Weights_norm:", x_weights_norm)

# print("Prices:", x_prices)
# print("Prices_Norm:", x_prices_norm)

# print("Capacity:", x_capacity)
# print("Best picks:", y_best_picks)

In [ ]:
# # Test mini-batch for testing the normalization
# print(features.shape)
# x_w = features[:,0,:]
# x_p = features[:,1,:]
# x_con = torch.cat([x_w, x_p], dim=1)
# print(x_con.shape)
# print(x_w[0])
# print(x_p[0])
# print(torch.mean(x_w[0]))

torch.Size([64, 2, 5])
torch.Size([64, 10])
tensor([0.0952, 0.2857, 0.1429, 0.1905, 0.2619])
tensor([0.1429, 0.7143, 0.4286, 1.0000, 0.5714])
tensor(0.1952)
